In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8 
    resonator_length_2.append(a)
print(resonator_length_2)


[7.653254683870284, 7.424788339841421, 7.2083465402091305]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-100}um",
                                              pos_y=f"{q0y+100}um",
                                              pad_width="150um",
                                              pad_height="60um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [19]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [20]:
hfss.start()

INFO 12:54PM [connect_project]: Connecting to Ansys Desktop API...
INFO 12:54PM [load_ansys_project]: 	Opened Ansys App
INFO 12:54PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 12:54PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project23
INFO 12:54PM [connect_design]: No active design found (or error getting active design).
INFO 12:54PM [connect]: 	 Connected to project "Project23". No design detected


True

In [21]:
hfss.activate_ansys_design("flip20_res7.4_coupler", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

12:54PM 51s WARNING [activate_ansys_design]: The design_name=flip20_res7.4_coupler was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 12:54PM [connect_design]: 	Opened active design
	Design:    flip20_res7.4_coupler [Solution type: Eigenmode]
WARNING 12:54PM [connect_setup]: 	No design setup detected.
WARNING 12:54PM [connect_setup]: 	Creating eigenmode default setup.
INFO 12:54PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [28]:
hfss.close()

True

# LOM Analysis

In [12]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.44',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.44',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [16]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 02:55PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp8ey4fzc4.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5440000000, Maxwell, 1, False
INFO 02:55PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmps9cg8be4.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5440000000, Maxwell, 1, False
INFO 02:55PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmprexjr7v9.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5440000000, Maxwell, 2, False
INFO 02:55PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpoxqhw4t1.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5440000000, Maxwell, 3, False
INFO 02:55PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp09l67t__.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5440000000, Maxwell, 4, False
INFO 02:55

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,283.700,-74.136000,-69.5560,-79.261000
pad_bot_Q0,-74.136,107.710000,-31.7220,-0.095694
pad_top_Q0,-69.556,-31.722000,107.4800,-4.423200
quad_coupler_c0,-79.261,-0.095694,-4.4232,83.819000


In [17]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.09
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.415230 [GHz]
EC 273.688919 [MHz]
EJ 14.854145 [GHz]
alpha -312.162928 [MHz]
dispersion 5.202506 [KHz]
Lq 10.995564 [nH]
Cq 70.774616 [fF]
T1 835.223940 [us]

**Coupling Properties**

tCqbus1 -2.167602 [fF]
gbus1_in_MHz -31.701900 [MHz]
χ_bus1 -0.192673 [MHz]
1/T1bus1 190.553617 [Hz]
T1bus1 835.223940 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.460843,278.600765,14.854145,-318.233174,6.218995,[-106.92811837182022],[-2.3411352282720785],2.341135,106.928118
2,5.706663,305.927223,14.854145,-352.287289,15.479624,[-76.65257841526184],[-1.7502315761270781],1.750232,76.652578
3,5.566585,290.177924,14.854145,-332.601895,9.299044,[-42.863733563102834],[-0.4400848454301805],0.440085,42.863734
4,5.582665,291.961852,14.854145,-334.823633,9.872299,[-39.23993636539207],[-0.37792949609844245],0.377929,39.239936
5,5.533205,286.494502,14.854145,-328.020964,8.203844,[-34.53133714563255],[-0.2716389046489545],0.271639,34.531337
6,5.481935,280.888746,14.854145,-321.066026,6.747105,[-31.83569302424489],[-0.21401695810091773],0.214017,31.835693
7,5.455855,278.061163,14.854145,-317.565557,6.099686,[-31.96335782548801],[-0.20768165734041288],0.207682,31.963358
8,5.43656,275.979664,14.854145,-314.991995,5.657457,[-31.889892908292836],[-0.20103432568538962],0.201034,31.889893
9,5.42308,274.530666,14.854145,-313.202095,5.365929,[-31.863542265884895],[-0.19684572023177882],0.196846,31.863542
10,5.41523,273.688919,14.854145,-312.162928,5.202506,[-31.701900491265057],[-0.19267254482000615],0.192673,31.701900


In [17]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
